In [16]:
import s3fs
import xarray as xr

In [17]:
bucket_src = "fcx-raw-data-temp"
# bucket_src = os.environ.get('SOURCE_BUCKET_NAME')
path_to_file="CRS/data"
# path_to_file = os.environ.get('PATH_TO_FEGS')
filename = "GOESR_CRS_L1B_20170517_v0.nc"
s3path = f"s3://{bucket_src}/{path_to_file}/{filename}"

In [18]:
s3path

's3://fcx-raw-data-temp/CRS/data/GOESR_CRS_L1B_20170517_v0.nc'

In [19]:
fs = s3fs.S3FileSystem(anon=True)

In [21]:
fs.ls("fcx-raw-data-temp")

['fcx-raw-data-temp/ABI',
 'fcx-raw-data-temp/AVIRIS',
 'fcx-raw-data-temp/CPL',
 'fcx-raw-data-temp/CRS',
 'fcx-raw-data-temp/FEGS',
 'fcx-raw-data-temp/GLM',
 'fcx-raw-data-temp/ISS_LIS',
 'fcx-raw-data-temp/LIP',
 'fcx-raw-data-temp/LMA',
 'fcx-raw-data-temp/NAV_ER2',
 'fcx-raw-data-temp/S-HIS',
 'fcx-raw-data-temp/S-HIS2']

In [32]:
with fs.open(s3path, 'rb') as f:
    DS = xr.load_dataset(f, engine="scipy", drop_variables=['lon', 'lat'])

print(DS)

<xarray.Dataset>
Dimensions:    (time: 74467, range: 321)
Coordinates:
  * time       (time) timedelta64[ns] 02:28:33.001154400 ... 07:35:23.641778400
  * range      (range) float32 1.012e+03 1.087e+03 ... 2.492e+04 2.5e+04
Data variables: (12/16)
    gatesp     float32 74.95
    missing    float32 nan
    incid      (time) float32 0.0 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
    roll       (time) float32 -1.341 -1.351 -1.307 ... -0.8147 -0.8147 -0.8147
    pitch      (time) float32 0.6765 0.7091 0.7446 ... -0.58 -0.5788 -0.5776
    track      (time) float32 -73.02 -73.02 -73.03 ... -74.68 -74.68 -74.68
    ...         ...
    wvel       (time) float32 5.708 5.718 5.711 ... -0.7386 -0.7345 -0.7305
    vacft      (time) float32 0.0 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
    pwr        (time, range) float32 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
    ref        (time, range) float32 -18.91 -21.36 -25.68 -17.94 ... nan nan nan
    dop        (time, range) float32 nan nan nan n

In [33]:
DS['ref']

<xarray.DataArray 'ref' (time: 74467, range: 321)>
array([[-18.905445, -21.356792, -25.679296, ...,        nan,        nan,
               nan],
       [-18.790478, -21.483028, -25.762894, ...,        nan,        nan,
               nan],
       [-18.927444, -21.16738 , -25.95508 , ...,        nan,        nan,
               nan],
       ...,
       [-40.323112, -36.766113, -38.658264, ...,        nan,        nan,
               nan],
       [-39.560387, -36.193375, -38.892174, ...,        nan,        nan,
               nan],
       [-38.19471 , -35.49832 , -38.108   , ...,        nan,        nan,
               nan]], dtype=float32)
Coordinates:
  * time     (time) timedelta64[ns] 02:28:33.001154400 ... 07:35:23.641778400
  * range    (range) float32 1.012e+03 1.087e+03 1.162e+03 ... 2.492e+04 2.5e+04
Attributes:
    long_name:  Radar Reflectivity
    units:      dBZ

In [35]:
DF = DS.to_dataframe()

In [39]:
DF

gatesp  missing         incid  \
time                      range                                            
0 days 02:28:33.001154400 1011.825012   74.949997      NaN  0.000000e+00   
                          1086.775024   74.949997      NaN  0.000000e+00   
                          1161.724976   74.949997      NaN  0.000000e+00   
                          1236.675049   74.949997      NaN  0.000000e+00   
                          1311.625000   74.949997      NaN  0.000000e+00   
...                                           ...      ...           ...   
0 days 07:35:23.641778400 24696.025391  74.949997      NaN  9.969210e+36   
                          24770.974609  74.949997      NaN  9.969210e+36   
                          24845.925781  74.949997      NaN  9.969210e+36   
                          24920.875000  74.949997      NaN  9.969210e+36   
                          24995.824219  74.949997      NaN  9.969210e+36   

                                            roll     pitch      track  \
time                      range                                         
0 days 02:28:33.001154400 1011.825012  -1.340926  0.676498 -73.015572   
                          1086.775024  -1.340926  0.676498 -73.015572   
                          1161.724976  -1.340926  0.676498 -73.015572   
                          1236.675049  -1.340926  0.676498 -73.015572   
                          1311.625000  -1.340926  0.676498 -73.015572   
...                                          ...       ...        ...   
0 days 07:35:23.641778400 24696.025391 -0.814730 -0.577582 -74.676399   
                          24770.974609 -0.814730 -0.577582 -74.676399   
                          24845.925781 -0.814730 -0.577582 -74.676399   
                          24920.875000 -0.814730 -0.577582 -74.676399   
                          24995.824219 -0.814730 -0.577582 -74.676399   

                                              height        head        evel  \
time                      range                                                
0 days 02:28:33.001154400 1011.825012   17494.628906  286.671875 -183.350861   
                          1086.775024   17494.628906  286.671875 -183.350861   
                          1161.724976   17494.628906  286.671875 -183.350861   
                          1236.675049   17494.628906  286.671875 -183.350861   
                          1311.625000   17494.628906  286.671875 -183.350861   
...                                              ...         ...         ...   
0 days 07:35:23.641778400 24696.025391  19721.119141  282.896454 -206.222198   
                          24770.974609  19721.119141  282.896454 -206.222198   
                          24845.925781  19721.119141  282.896454 -206.222198   
                          24920.875000  19721.119141  282.896454 -206.222198   
                          24995.824219  19721.119141  282.896454 -206.222198   

                                             nvel      wvel         vacft  \
time                      range                                             
0 days 02:28:33.001154400 1011.825012   56.001507  5.708407  0.000000e+00   
                          1086.775024   56.001507  5.708407  0.000000e+00   
                          1161.724976   56.001507  5.708407  0.000000e+00   
                          1236.675049   56.001507  5.708407  0.000000e+00   
                          1311.625000   56.001507  5.708407  0.000000e+00   
...                                           ...       ...           ...   
0 days 07:35:23.641778400 24696.025391  56.507305 -0.730452  9.969210e+36   
                          24770.974609  56.507305 -0.730452  9.969210e+36   
                          24845.925781  56.507305 -0.730452  9.969210e+36   
                          24920.875000  56.507305 -0.730452  9.969210e+36   
                          24995.824219  56.507305 -0.730452  9.969210e+36   

                                                 pwr        r

In [44]:
DF[['ref', 'roll']]

ref      roll
time                      range                            
0 days 02:28:33.001154400 1011.825012  -18.905445 -1.340926
                          1086.775024  -21.356792 -1.340926
                          1161.724976  -25.679296 -1.340926
                          1236.675049  -17.936014 -1.340926
                          1311.625000  -16.003502 -1.340926
...                                           ...       ...
0 days 07:35:23.641778400 24696.025391        NaN -0.814730
                          24770.974609        NaN -0.814730
                          24845.925781        NaN -0.814730
                          24920.875000        NaN -0.814730
                          24995.824219        NaN -0.814730

[23903907 rows x 2 columns]